<a href="https://colab.research.google.com/github/VGODIE/ML_kaggle_competitions/blob/master/Analysis_of_new_build_housing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Результат работы

Результатом работы является сводная таблица, имеющая id дома и его характеристики: Регион, Застройщик, Квартал сдачи, Год сдачи, Банк-кредитор, Общее количество квартир, общее количество квадратных метров, наличие кредита на стройку, наличие эксроу-счетов, общая стоимость квартир, средняя стоимость одной квартиры, количество проданных квартир, суммарная стоимость проданных квартир 

# План работы и время на ее выполнение

Эта задача сбора данных в моем понимании может быть разбита на 3 последующих крупных этапа работы:

1. Создание базового корректного парсера и модели обработки данных

Цель этого этапа: Создать исходную модель, которая будет корректно обрабатывать все данные, проходя все шаги решения - то есть выстроить каркас

2. Оптимизация парсера

Цель: Максимизация доли корректно спарсенных и обработанных данных за счет учета большего разнообразия случаев и специфики данных - то есть наполнение каркаса содержимым для усовершенствования качества собранных данных

3. Оптимизация скорости работы парсера

Цель: Видоизменить модель таким образом, чтобы время ее выполнения соответствовало установленным показателям. 


На каждый из этапов работы я бы выделил по **1 неделе**: итого спланировав весь проект сделать за 3 недели


# Карта данных

Для того, чтобы получить эту информацию необходимо распарсить несколько ключевых источников:

Основным ресурсом является сайт ЕИСЖС. А конкретно из него необходимы несколько видов страниц.

Первым из них является каталог новостроек ![alt text](https://drive.google.com/uc?id=11-b7EOJsH_exE-Hw1_2mkm02Fv7eHABg)
Переходя на отдельные страницы каждого дома, можно спарсить населенный пункт, застройщика, наличие эскроу, количество этажей и квартир, квартал и год сдачи, а также общую жилую площадь (выделено фиолетовым)
![alt text](https://drive.google.com/uc?id=1AtbYYb9rPsVUqJ6ZFNabFDI5PkJwu412)



Вторым важным источником являются проектные декларации, на которые можно перейти со страницы, показанной выше

Из этих деклараций можно спарсить банка-кредитора, если он есть, ,банк, в котором должны открываться счета эскроу

А кроме этого, из деклараций можно спарсить суммарное количество договоров, суммарную площадь и суммарную стоимость проданных квартир для определения следующих величин:
1. Примерное количество проданных квартир

Такой информации напрямую нет, но ее можно оценить, сначала получив среднюю площадь одной квартиры: общая площадь / общее количество квартир

А затем, чтобы оценить примерно количество проданных квартир нужно: площадь проданных квартир / среднюю площадь одной квартиры
2. Средняя стоимость одной квартиры и как следствие суммарная стоимость всех существующих квартир

Такой информации тоже напрямую нет, но ее можно оценить следующим образом:

Средняя стоимость квартиры это суммарная стоимость проданных квартир / количество проданных квартир

Тогда суммарная стоимость всех квартир это количество квартир * средняя стоимость одной квартиры
 ![alt text](https://drive.google.com/uc?id=1mo16JdVPl2jV1UPQRl3vTZkXPWVwsydf)

 ![alt text](https://drive.google.com/uc?id=1WgEn8DsunM048FBigAaO9q3-VgHxJ5NF)

 ![alt text](https://drive.google.com/uc?id=1TR-Z1itBoXkx7y6EWerR_7ouHHWobZvw)

 ![alt text](https://drive.google.com/uc?id=1hN2yULk7pwLqkGxanDZznJoXvLz4IQ92)





# Алгоритм работы базового алгоритма


0. Для разбиения по регионам я бы, в первую очередь, спарсил ссылки на списки новостроек по регионам, чтобы парсить их
последовательно и когда объекты одного региона заканчиваются, то всем спарсенным присваивать этот регион

1. После того как парсер перейдет на страницу конкретного региона в каталог новостроек, то парсим ссылки на страницы конкретных объектов
и ссылку на следующую страницу. Затем интеративно обрабатываем каждую из ссылок объектов, закончив, переходим на следующую страницу
объектов и так пока не закончатся объекты данного региона. А после перехода на новый регион итеративно потворяем те же операции.

2. На каждой странице парсим те данные, о которых я писал в предыдущем разделе, а также парсим ссылку на проектную декларацию
данного объекта и храним ее как отдельный признак объекта - обработаем ее позже, так как она представлена в pdf.
Каждую характеристку складываем в отдельный список - эти списки сконкатенируем в матрицу после парсинга всего региона

3. После парсинга всех объектов, достаем из полученного датасета ссылки на pdf файлы с проектными декларациями. Итерируясь по ним, парсим с них данные, описанные в предыдущем пункте и добавляем их в датасет.

4. После получения всех спарсенных данных в виде таблицы необходимо получить признаки, которые являются производными от спарсенных,
к примеру как общая стоимость квартир в доме или количество проданных квартир. А также в этом шаге необходимо сделать постобработку
всех текстовых признаков - к примеру если рассматривать банк, где открываются эскроу, то тот же Сбербанк там может фигурировать как
как филиал такого-то отделения - такие данные необходимо привести к единому виду. Подобную обработку нужно будет сделать для всех текстовых признаков, которые в исходном источнике данных могут быть записаны в несколько различных формах (Это можно делать и в процессе парсинга в процессе оптимизации скорости работы парсера, если это будет целесообразно)

5. Моя оценка общей стоимости квартир имеет минусы - один из них заключается в том, что в домам, которые будут сданы более чем через 10 лет бывает, что нет еще купленных квартир, как следствие на основе моего подхода не получается оценить стоимость одной квартиры и как следствие суммарную стоимость всех квартир. Решением этой проблемы, которое я предлагаю, это обучить ML-модель для предсказания этих параметров на основе размеченных данных, которые удалось корректно спарсить.

# Инструментарий

Для выполнения цели потребуется использование следующих библиотек:

1. BeautifulSoup (bs4), а также  scrapy, requests для парсинга веб-страниц
2. Pandas для представления данных в виде датасета и преобразование их
3. Numpy для эффективной работы с матрицами: а именно для преобразования численных признаков
4. Re и NLTK для обработки конкрентных текстовых объектов
5. Sklearn для создания ML-модели для восстановления данных, которые пропущены на основе размещенных данных

# Пример парсинга страницы объекта

In [0]:
import bs4 as bs
import urllib.request
import requests

In [0]:
r = requests.get('https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/%D0%BA%D0%B0%D1%82%D0%B0%D0%BB%D0%BE%D0%B3-%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BA/%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82/34120')

In [0]:
soup = bs.BeautifulSoup(r.content)

In [0]:
def parse_page(soup):
  house_id = soup.find('p',{"class":"styles__Id-eng632-2 liGaEP"}).text
  adress = soup.find('p',{"class":"styles__Id-eng632-2 styles__Address-eng632-4 bVtIlE"}).text
  builder = soup.find('a',{"class":"styles__LinkContainer-sc-1u7ca6h-0 gxEzAu"}).text
  x = soup.find('p',{"class":"styles__StatusText-pg2ty1-1 gkUmPu"}).text
  n_floors = soup.findAll('div',{"class":"styles__Value-sc-13pfgqd-2 eTcBsy"})[2].text
  n_appartments = soup.findAll('div',{"class":"styles__Value-sc-13pfgqd-2 eTcBsy"})[3].text
  exp_start = soup.findAll('div',{"class":"styles__Value-sc-13pfgqd-2 eTcBsy"})[4].text
  area = soup.findAll('p',{"class":"styles__Value-sc-2i4wxn-3 jsrVhl"})[-1].text
  res = [house_id, adress, builder, x, n_floors, n_appartments, exp_start, area]
  return res

In [85]:
parse_page(soup)

['ID дома: 34120',
 'Адрес: Москва город, г Москва, район Южнопортовый, пр-кт Волгоградский, корпус Б, вл. 32/5',
 'ООО СЗ  МЕТРОПОЛИЯ',
 'Продажа квартир через счета эскроу',
 '32',
 '522',
 'II кв. 2025',
 '27 032']